In [1]:
from data_processing import txt_file_path_loader

In [2]:
# create a training, testing and validation list containing relevant file paths
test_file_path = './data/testing_list.txt'
val_file_path = './data/validation_list.txt'
base_dir = './data/'

test_paths = txt_file_path_loader(test_file_path, base_dir)
val_paths = txt_file_path_loader(val_file_path, base_dir)